# Lab 12: Grid Localization using Bayes Filter (Real Robot)

### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

### <span style="color:rgb(0,150,0)">The notebook only provides skeleton code for you to integrate the Localization class with the Real Robot.</span>

<hr>

In [1]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio
import pathlib
import os
from utils import load_config_params
from localization_extras import Localization

# The imports below will only work if you copied the required ble-related python files 
# into the notebooks directory
from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import numpy as np

# Setup Logger
LOG = get_logger('demo_notebook.log')
LOG.propagate = False

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

2022-05-01 17:40:31,422 | INFO     |: Logger demo_notebook.log initialized.


In [2]:
# Start the plotter
START_PLOTTER()

# The RealRobot class
Define the RealRobot class in the code cell below, based on the documentation and your real robot communication protocol. <br>
This class is used by the **Localization** class to communicate with the real robot. <br>
More specifically, the **Localization** class utilizes the **RealRobot's** member function **perform_observation_loop()** to get the 18 sensor readings and store them in its member variable **obs_range_data**, which is then utilized in the update step.

In [3]:
class RealRobot():
    """A class to interact with the real robot
    """
    def __init__(self, commander, ble):
        # Load world config
        self.world_config = os.path.join(str(pathlib.Path(os.getcwd()).parent), "config", "world.yaml")
        
        self.config_params = load_config_params(self.world_config)
        
        # Commander to commuincate with the Plotter process
        # Used by the Localization module to plot odom and belief
        self.cmdr = commander

        # ArtemisBLEController to communicate with the Robot
        self.ble = ble

        # Data collection stuff
        self.dist_data = []
        self.angle_data = []

        self.setup_notify()
        

    def setup_notify(self):
        print("Notifications started")
        ble.start_notify(ble.uuid['RX_TOF1'], self.tof1_callback_handler)
        print(ble.uuid['RX_TOF1'])
        ble.start_notify(ble.uuid['RX_IMU_PITCH'], self.imu_pitch_callback_handler)

    def tof1_callback_handler(self, uuid, byte_array):
        new_tof = self.ble.bytearray_to_float(byte_array)
        print(f"Got TOF data: {new_tof}")
        self.dist_data.append(new_tof)

    def imu_pitch_callback_handler(self, uuid, byte_array):
        new_angle = self.ble.bytearray_to_float(byte_array)
        print(f"Got angle data: {new_angle}")
        self.angle_data.append( new_angle )
        
    def set_params(self, pid_p, pid_min_power, imu_hz):
        self.ble.send_command(CMD.SET_PID, f"{pid_p}|{pid_min_power}|{imu_hz}"); 

    def set_motor_calib(self, calib):
        self.ble.send_command(CMD.SET_MOTOR_CALIB, calib)
        
    def get_pose(self):
        """Get robot pose based on odometry
        
        Returns:
            current_odom -- Odometry Pose (meters, meters, degrees)
        """
        raise NotImplementedError("get_pose is not implemented")

    def perform_observation_loop(self, rot_vel=120):
        """Perform the observation loop behavior on the real robot, where the robot does  
        a 360 degree turn in place while collecting equidistant (in the angular space) sensor
        readings, with the first sensor reading taken at the robot's current heading. 
        The number of sensor readings depends on "observations_count"(=18) defined in world.yaml.
        
        Keyword arguments:
            rot_vel -- (Optional) Angular Velocity for loop (degrees/second)
                        Do not remove this parameter from the function definition, even if you don't use it.
        Returns:
            sensor_ranges   -- A column numpy array of the range values (meters)
            sensor_bearings -- A column numpy array of the bearings at which the sensor readings were taken (degrees)
                               The bearing values are not used in the Localization module, so you may return a empty numpy array
        """
        self.set_params(1, 80, 50)
        self.set_motor_calib(1.4)
        self.ble.send_command(CMD.OBSERVE, None)
        self.ble.send_command(CMD.GET_DATA, None)
        while len(self.dist_data) < 18 or len(self.angle_data) < 18:
            print("Data still sending, waiting")
            time.sleep(5)
            self.ble.send_command(CMD.GET_DATA, None) # try again for command not sent?
        self.dist_data = [x / 1000 for x in self.dist_data]
        temp_angles = []
        temp_sum = 0
        for i,angle in enumerate(self.angle_data):
            temp_sum -= angle
            temp_angles.append(temp_sum)
        self.angle_data = temp_angles
        print(self.dist_data)
        print(self.angle_data)
        return np.array(self.dist_data).reshape((18,1)), np.array(self.angle_data).reshape((18,1))


In [4]:
# # Get ArtemisBLEController object
# ble = get_ble_controller()

# # Connect to the Artemis Device
# ble.connect()

In [7]:
# Initialize RealRobot with a Commander object to communicate with the plotter process
# and the ArtemisBLEController object to communicate with the real robot
# robot = RealRobot(cmdr, None)
robot = VirtualRobot(cmdr)
# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a RealRobot object and a Mapper object as parameters
loc = Localization(robot, mapper)

## Plot Map
cmdr.plot_map()

2022-05-01 17:41:28,417 | INFO     |:  | Number of observations per grid cell: 18
2022-05-01 17:41:28,424 | INFO     |:  | Precaching Views...


/Users/main/Documents/Classes/2022-sp/ECE-4960/ece4960-fast-robots-code/lab10/ECE4960-sim-release-release-mirror/localization.py:150: RuntimeWarning: All-NaN slice encountered
  return np.nanmin(distance_intersections_tt), intersections_tt[np.nanargmin(distance_intersections_tt)]


2022-05-01 17:41:32,322 | INFO     |:  | Precaching Time: 3.897 secs
2022-05-01 17:41:32,323 | INFO     |: Initializing beliefs with a Uniform Distribution
2022-05-01 17:41:32,324 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107


# Run an update step of the Bayes Filter

In [23]:
# Reset Plots
cmdr.reset_plotter()

def ft2m(pt):
    m = 0.3048
    return (pt[0] * m, pt[1] * m)

def m2ft(pt):
    m = 3.28084
    return (pt[0] * m, pt[1] * m)

gt = (0,3)
gt = ft2m(gt)

bel = (-0.305,-0.914)
cmdr.plot_bel(bel[0], bel[1])
print(m2ft(bel))

bel = (1.524,0.305)
cmdr.plot_bel(bel[0], bel[1])
print(m2ft(bel))

bel = (0.914,-0.914)
cmdr.plot_bel(bel[0], bel[1])
print(m2ft(bel))

cmdr.plot_gt(gt[0], gt[1])

(-1.0006561999999999, -2.99868776)
(5.00000016, 1.0006561999999999)
(2.99868776, -2.99868776)


# 